In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# SubproblemComp

`SubproblemComp` provides a way to evaluate an OpenMDAO system within an OpenMDAO system.

# SubproblemComp Options

In [2]:
import openmdao.api as om

submodel1 = om.Group()
submodel1.add_subsystem('subComp1', om.ExecComp('x = r*cos(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])

om.show_options_table(om.SubproblemComp(model=submodel1, inputs=['r', 'theta'],
                          outputs=['x']))

Option,Default,Acceptable Values,Acceptable Types,Description
distributed,False,"[True, False]",['bool'],"If True, set all variables in this component as distributed across multiple processes"
inputs,{},N/A,['dict'],Subproblem Component inputs
outputs,{},N/A,['dict'],Subproblem Component outputs
run_root_only,False,"[True, False]",['bool'],"If True, call compute, compute_partials, linearize, apply_linear, apply_nonlinear, and compute_jacvec_product only on rank 0 and broadcast the results to the other ranks."


# SubproblemComp Constructor

The call signature for the `SubproblemComp` constructor is

```{eval-rst}
    .. automethod:: openmdao.components.subproblem_comp.SubproblemComp.__init__
        :noindex:
```

# Using the SubproblemComp

`SubproblemComp` allows you to add a component that is a `System` itself. Its required arguments are `model`, `inputs`, and `outputs`. Other optional arguments are arguments you can use to configure the `Problem` within the subproblem (see [problem.py](../../../_srcdocs/packages/core/problem.html#openmdao.core.problem.Problem)). The only difference is, the kwargs used for `Problem` have to be passed into `SubproblemComp` as a `dict` rather than as kwargs.

The arg that stores the kwargs for `Problem` is `prob_options`. Other kwargs for `ExplicitComponent` can then be passed normally to `SubproblemComp`.

## IO Format

Inputs and outputs can be passed into `SubproblemComp` as a list of `str`, `tuple`, or both. If the list element is `str`, then it must be the group level promoted name that is passed. For this, the IO to be referred to must be promoted in `add_subsystem`. If the list element is `tuple`, then the first element of the `tuple` must be the group level promoted name or the absolute name of the variable, and the second element is the alias that `SubproblemComp` will refer to it as.

e.g. (path.to.var, desired_var_name)  
or (var, desired_var_name) if the variable is already promoted at the group level

Input and output names can be wildcards too, only if their type is `str`.


# Example: Implementing SubproblemComp

In [2]:
import openmdao.api as om
from numpy import pi

p = om.Problem()

model = om.Group()
model.add_subsystem('supComp', om.ExecComp('z = x**2 + y'),
                    promotes_inputs=['x', 'y'],
                    promotes_outputs=['z'])

submodel1 = om.Group()
submodel1.add_subsystem('sub1_ivc_r', om.IndepVarComp('r', 1.),
                        promotes_outputs=['r'])
submodel1.add_subsystem('sub1_ivc_theta', om.IndepVarComp('theta', pi),
                        promotes_outputs=['theta'])
submodel1.add_subsystem('subComp1', om.ExecComp('x = r*cos(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['x'])

submodel2 = om.Group()
submodel2.add_subsystem('sub2_ivc_r', om.IndepVarComp('r', 2),
                        promotes_outputs=['r'])
submodel2.add_subsystem('sub2_ivc_theta', om.IndepVarComp('theta', pi/2),
                        promotes_outputs=['theta'])
submodel2.add_subsystem('subComp2', om.ExecComp('y = r*sin(theta)'),
                        promotes_inputs=['r', 'theta'],
                        promotes_outputs=['y'])

subprob1 = om.SubproblemComp(model=submodel1, inputs=['r', 'theta'],
                            outputs=['x'])
subprob2 = om.SubproblemComp(model=submodel2, inputs=['r', 'theta'],
                            outputs=['y'])

p.model.add_subsystem('sub1', subprob1, promotes_inputs=['r','theta'],
                            promotes_outputs=['x'])
p.model.add_subsystem('sub2', subprob2, promotes_inputs=['r','theta'],
                            promotes_outputs=['y'])
p.model.add_subsystem('supModel', model, promotes_inputs=['x','y'],
                            promotes_outputs=['z'])

p.setup(force_alloc_complex=True)

p.set_val('r', 1)
p.set_val('theta', pi)

p.run_model()
cpd = p.check_partials(method='cs')

print(f"r = {p.get_val('r')}")
print(f"theta = {p.get_val('theta')}")
print(f"x = {p.get_val('x')}")
print(f"y = {p.get_val('y')}")
print(f"z = {p.get_val('z')}")

--------------------------------
Component: SubproblemComp 'sub1'
--------------------------------

  sub1: 'x' wrt 'r'
    Analytic Magnitude: 1.000000e+00
          Fd Magnitude: 1.000000e+00 (cs:None)
    Absolute Error (Jan - Jfd) : 0.000000e+00

    Relative Error (Jan - Jfd) / Jfd : 0.000000e+00

    Raw Analytic Derivative (Jfor)
[[-1.]]

    Raw CS Derivative (Jcs)
[[-1.]]
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
  sub1: 'x' wrt 'theta'
    Analytic Magnitude: 1.224647e-16
          Fd Magnitude: 1.224647e-16 (cs:None)
    Absolute Error (Jan - Jfd) : 0.000000e+00

    Relative Error (Jan - Jfd) / Jfd : 0.000000e+00

    Raw Analytic Derivative (Jfor)
[[-1.2246468e-16]]

    Raw CS Derivative (Jcs)
[[-1.2246468e-16]]
 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
--------------------------------
Component: SubproblemComp 'sub2'
--------------------------------

  sub2: 'y' wrt 'r'
    Analytic Magnitude: 1.224647e-16
          Fd Magnitude: 1.2

# Example: Variable Aliases

In [3]:
import openmdao.api as om

p = om.Problem()
model = om.Group()

model.add_subsystem('subsys', om.ExecComp('z = x**2 + y**2'))
subprob = om.SubproblemComp(model=model, inputs=[('subsys.x', 'a'), ('subsys.y', 'b')],
                            outputs=[('subsys.z', 'c')])

p.model.add_subsystem('subprob', subprob, promotes_inputs=['a', 'b'], promotes_outputs=['c'])
p.setup()

p.set_val('a', 1)
p.set_val('b', 2)

p.run_model()

print(f"a = {p.get_val('a')}")
print(f"b = {p.get_val('b')}")
print(f"c = {p.get_val('c')}")

2 Input(s) in 'subprob'

varname  val 
-------  ----
a        [1.]
b        [2.]


1 Explicit Output(s) in 'subprob'

varname  val 
-------  ----
c        [5.]


0 Implicit Output(s) in 'subprob'


a = [1.]
b = [2.]
c = [5.]


# Example: Using Variables with the Same Name

In [4]:
import openmdao.api as om

p = om.Problem()

model = om.Group()

model.add_subsystem('x1Comp', om.ExecComp('x1 = x*3'))
model.add_subsystem('x2Comp', om.ExecComp('x2 = x**3'))
model.connect('x1Comp.x1', 'model.x1')
model.connect('x2Comp.x2', 'model.x2')
model.add_subsystem('model', om.ExecComp('z = x1**2 + x2**2'))

subprob = om.SubproblemComp(model=model, inputs=[('x1Comp.x', 'x'), ('x2Comp.x', 'y')],
                            outputs=[('model.z', 'z')])

p.model.add_subsystem('subprob', subprob)
p.setup()

p.set_val('subprob.x', 1)
p.set_val('subprob.y', 2)

p.run_model()

print(f"x1Comp.x = {p.get_val('subprob.x')}")
print(f"x2Comp.x = {p.get_val('subprob.y')}")
print(f"model.z = {p.get_val('subprob.z')}")

x1Comp.x = [1.]
x2Comp.x = [2.]
model.z = [73.]
